In [21]:
import pandas as pd
import numpy as np

In [22]:
### emission data
# http://cdiac.ornl.gov/ftp/ndp030/CSV-FILES/
raw = pd.read_excel('indicator CDIAC carbon_dioxide_cumulative_emissions.xlsx')
raw = raw[raw.iloc[:,1:].any(axis=1)]
raw.rename(columns={'CO2 emissions from fossil-fuels since 1751 (metric tons)':'name'}, inplace=True)

In [23]:
import pycountry
def rename(s):
    try:
        numeric_id=pycountry.countries.get(name=s).numeric
    except:
        numeric_id = None
    return numeric_id

raw['name'].map(rename).isnull().sum()

45

In [24]:
pop = pd.read_csv('population.csv',skiprows=3)[['Country Name','Country Code','2014']]
pop.columns = ['Country Name','Country Code','population']

In [25]:
def get_id(s):
    #s = "{0:0>3}".format(n)
    try:
        numeric_id=int(pycountry.countries.get(alpha3=s).numeric)
    except:
        numeric_id = -1
    return numeric_id

pop['id'] = pop['Country Code'].map(get_id)
pop = pop[pop['id']!=-1]

In [26]:
area = pd.read_csv('area.csv',skiprows=3)[['Country Name','Country Code','2015']]
area.columns = ['Country Name','Country Code','area']
area['id'] = area['Country Code'].map(get_id)
area = area[area['id']!=-1]

In [27]:
nm_df = pd.read_table('world-country-names.tsv')
nm_dic = {'Bolivia, Plurinational State of': 'Bolivia',
          'Virgin Islands, British': 'British Virgin Islands',
          'Brunei Darussalam': 'Brunei',
          'Congo': 'Congo, Rep.',
          'Congo, the Democratic Republic of the': 'Congo, Dem. Rep.',
          'Cook Islands': 'Cook Is',
          "Côte d'Ivoire": "Cote d'Ivoire",
          'Faroe Islands': 'Faeroe Islands',
          'Falkland Islands (Malvinas)': 'Falkland Is (Malvinas)',
          'Hong Kong': 'Hong Kong, China',
          'Iran, Islamic Republic of': 'Iran',
          "Korea, Democratic People's Republic of": 'North Korea',
          'Korea, Republic of': 'South Korea',
          'Kyrgyzstan': 'Kyrgyz Republic',
          "Lao People's Democratic Republic": "Lao",
          'Macao': 'Macao, China',
          'Macedonia, the former Yugoslav Republic of': 'Macedonia, FYR',
          'Micronesia, Federated States of': 'Micronesia, Fed. Sts.',
          'Moldova, Republic of': 'Moldova',
          'Réunion': 'Reunion',
          'Russian Federation': 'Russia',
          'Saint Helena, Ascension and Tristan da Cunha': 'St. Helena',
          'Saint Kitts and Nevis': 'St. Kitts and Nevis',
          'Saint Lucia': 'St. Lucia',
          'Saint Vincent and the Grenadines': 'St. Vincent and the Grenadines',地
          'Saint Pierre and Miquelon': 'St.-Pierre-et-Miquelon',
          'Slovakia': 'Slovak Republic',
          'Syrian Arab Republic': 'Syria',
          'Taiwan, Province of China': 'Taiwan',
          'Tanzania, United Republic of': 'Tanzania',
          'Wallis and Futuna': 'Wallis et Futuna',
          'Venezuela': 'Venezuela, Bolivarian Republic of',
          'Viet Nam': 'Vietnam',
          'Bonaire, Sint Eustatius and Saba': 'Saba'}

nm_df['name'] = nm_df['name'].map(lambda s: nm_dic[s] if s in nm_dic.keys() else s)

In [28]:
nm_df = nm_df.merge(pop[['id','population']], how='left', left_on='id', right_on='id')
nm_df = nm_df.merge(area[['id','area']], how='left', left_on='id', right_on='id')

In [29]:
df = nm_df.merge(raw, how='left', left_on='name', right_on='name')

In [30]:
df[['population','area']] = df[['population','area']].fillna(-1)
df = df[(df['id']>0)&(df['population']>0)&(df['area']>0)]

In [37]:
agg_idx = max(df.index)+1
#df.loc[agg_idx,:] = np.nan
df.loc[agg_idx,'id'] = -1
df.loc[agg_idx,'name'] = 'World'
df.iloc[-1,2:] = df.iloc[:-1,2:].sum()

In [41]:
df['id'] = df['id'].astype(int)

In [44]:
df

,id,name,population,area,1751,1755,1760,1765,1770,1775,...,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
3,4,Afghanistan,3.162751e+07,652860.0,NaN,NaN,NaN,NaN,NaN,NaN,...,7.326000e+07,7.384300e+07,7.457633e+07,7.557367e+07,7.684600e+07,7.873433e+07,8.251100e+07,8.882500e+07,NaN,NaN
4,8,Albania,2.894475e+06,27400.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2.055863e+08,2.098763e+08,2.139903e+08,2.185920e+08,2.228307e+08,2.273223e+08,2.314510e+08,2.344577e+08,NaN,NaN
6,12,Algeria,3.893433e+07,2381740.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2.305772e+09,2.398297e+09,2.487782e+09,2.594900e+09,2.702194e+09,2.814926e+09,2.926220e+09,3.047521e+09,3.165537e+09,3.289835e+09
7,16,American Samoa,5.543400e+04,200.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,20,Andorra,7.278600e+04,470.0,NaN,NaN,NaN,NaN,NaN,NaN,...,3.868333e+06,4.403667e+06,4.968333e+06,5.544000e+06,6.090333e+06,6.629333e+06,7.168333e+06,7.685333e+06,NaN,NaN
9,24,Angola,2.422752e+07,1246700.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2.111817e+08,2.202457e+08,2.390373e+08,2.581920e+08,2.804560e+08,3.056057e+08,3.316280e+08,3.582810e+08,NaN,NaN
10,28,Antigua and Barbuda,9.090000e+04,440.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.459333e+07,1.498200e+07,1.538900e+07,1.579967e+07,1.622500e+07,1.666133e+07,1.710867e+07,1.757067e+07,NaN,NaN
11,31,Azerbaijan,9.535079e+06,82659.0,NaN,NaN,NaN,NaN,NaN,NaN,...,3.926340e+08,4.232470e+08,4.553340e+08,4.896687e+08,5.288323e+08,5.743723e+08,6.273043e+08,6.763753e+08,7.240623e+08,7.753676e+08
12,32,Argentina,4.298003e+07,2736690.0,NaN,NaN,NaN,NaN,NaN,NaN,...,5.221627e+09,5.354741e+09,5.510897e+09,5.666533e+09,5.837696e+09,6.011269e+09,6.193381e+09,6.368083e+09,6.549688e+09,6.745246e+09
13,36,Australia,2.347012e+07,7682300.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.153038e+10,1.187682e+10,1.222602e+10,1.259312e+10,1.297046e+10,1.335179e+10,1.374475e+10,1.414491e+10,1.450681e+10,1.489895e+10


In [42]:
data_dict = []

for idx, row in df.iterrows():
    datum = {}
    datum['id'] = row['id']
    datum['name'] = row['name']
    datum['population'] = row['population']
    datum['area'] = row['area']
    data = row.iloc[4:].dropna()
    if data.any():
        cumulative = {}
        annual = {}
        
        last = 0
        for year in data.index:
            current = data[year]
            cumulative[year] = current
            if current - last:
                annual[year] = current - last
            last = current
        datum['annual'] = annual
        datum['cumulative'] = cumulative
    data_dict.append(datum)

In [43]:
import json
with open('emission.json', 'w') as outfile:
    json.dump(data_dict, outfile)